In [ ]:
# refacto V3 :
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import csv

ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}


def extraire_donnees_v3(fichier_xml: str) -> pd.DataFrame:
    try:
        tree = ET.parse(fichier_xml)
        root = tree.getroot()
    except Exception as e:
        print(f"❌ Erreur de parsing {fichier_xml} : {e}")
        return pd.DataFrame()

    def get_text(path):
        elem = root.find(path, ns)
        return elem.text.strip() if elem is not None and elem.text else ""

    # Métadonnées
    metadata = {
        "UID": get_text(".//ns:uid"),
        "SeanceRef": get_text(".//ns:seanceRef"),
        "SessionRef": get_text(".//ns:sessionRef"),
        "DateSeance": get_text(".//ns:dateSeance"),
        "DateSeanceJour": get_text(".//ns:dateSeanceJour"),
        "NumSeanceJour": get_text(".//ns:numSeanceJour"),
        "NumSeance": get_text(".//ns:numSeance"),
        "TypeAssemblee": get_text(".//ns:typeAssemblee"),
        "Legislature": get_text(".//ns:legislature"),
        "Session": get_text(".//ns:session"),
        "NomFichierJO": get_text(".//ns:nomFichierJo"),
        "President": get_text(".//ns:presidentSeance"),
    }

    rows = []
    titre_general = ""
    sous_titre = ""

    contenu = root.find("ns:contenu", ns)
    if contenu is None:
        return pd.DataFrame()

    for elem in contenu.iter():
        tag = elem.tag.split("}")[-1]
        code_grammaire = elem.attrib.get("code_grammaire", "")
        code_style = elem.attrib.get("code_style", "")

        if code_grammaire == "TITRE_TEXTE_DISCUSSION" and code_style == "Titre":
            texte = elem.find("ns:texte", ns)
            titre_general = (
                texte.text.strip() if texte is not None and texte.text else ""
            )

        elif (
            code_grammaire == "SOUS_TITRE_TEXTE_DISCUSSION"
            and code_style == "Sous-tit_info"
        ):
            texte = elem.find("ns:texte", ns)
            sous_titre = texte.text.strip() if texte is not None and texte.text else ""

        elif tag == "paragraphe":
            texte_elem = elem.find("ns:texte", ns)
            if texte_elem is None:
                continue

            texte = "".join(texte_elem.itertext()).strip()
            stime = texte_elem.attrib.get("stime", "")

            orateur_elem = elem.find(".//ns:orateur", ns)
            nom, qualite, id_orateur = "", "", ""
            if orateur_elem is not None:
                nom = orateur_elem.find("ns:nom", ns)
                qualite = orateur_elem.find("ns:qualite", ns)
                id_elem = orateur_elem.find("ns:id", ns)

                nom = nom.text.strip() if nom is not None and nom.text else ""
                qualite = (
                    qualite.text.strip() if qualite is not None and qualite.text else ""
                )
                id_orateur = (
                    id_elem.text.strip() if id_elem is not None and id_elem.text else ""
                )

            rows.append(
                {
                    **metadata,
                    "Titre_general": titre_general,
                    "Sous_titre": sous_titre,
                    "ID_paragraphe": elem.attrib.get("id_syceron", ""),
                    "Ordre_seance": elem.attrib.get("ordre_absolu_seance", ""),
                    "Code_grammaire": code_grammaire,
                    "Code_style": code_style,
                    "Code_parole": elem.attrib.get("code_parole", ""),
                    "Role_debat": elem.attrib.get("roledebat", ""),
                    "Nom_orateur": nom,
                    "Qualite_orateur": qualite,
                    "ID_orateur": id_orateur,
                    "stime": stime,
                    "Texte": texte,
                    "Fichier_source": os.path.basename(fichier_xml),
                }
            )

    return pd.DataFrame(rows)


def traiter_dossier_v3(dossier_path: str, pattern="*.xml", output_csv="export_v3.csv"):
    fichiers = glob.glob(os.path.join(dossier_path, pattern))
    if not fichiers:
        print("❌ Aucun fichier XML trouvé.")
        return

    print(f"🔍 Traitement de {len(fichiers)} fichiers XML dans le dossier...")

    dfs = []
    for fichier in fichiers:
        print(f"➡️  {os.path.basename(fichier)}")
        df = extraire_donnees_v3(fichier)
        if not df.empty:
            dfs.append(df)

    if dfs:
        df_concat = pd.concat(dfs, ignore_index=True)
        df_concat.to_csv(
            output_csv, index=False, quoting=csv.QUOTE_ALL, encoding="utf-8"
        )
        print(f"\n✅ Export CSV : {output_csv} ({df_concat.shape[0]} lignes)")
    else:
        print("⚠️ Aucun paragraphe extrait.")


# traiter_dossier_v3("../data/16-xml/compteRendu/")

In [ ]:
# V4
# Refactorisation claude
# Plus d'informations (contexte des points)
# Traçabilité (source d'extraction)

# Essai refactorisation claude
import pandas as pd
import xml.etree.ElementTree as ET
import csv
import os
import glob


def extraire_donnees_assemblee(fichier_xml: str) -> pd.DataFrame:
    """
    Extrait les données d'un fichier XML de compte rendu de l'Assemblée nationale
    Version hybride : combine contexte global ET informations des points ODJ.
    """

    # Charger le XML
    tree = ET.parse(fichier_xml)
    root = tree.getroot()
    ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}

    def get_text(path):
        """Fonction helper pour extraire le texte de manière sûre"""
        elem = root.find(path, ns)
        return elem.text.strip() if elem is not None and elem.text else ""

    # Métadonnées générales
    metadata = {
        "UID": get_text(".//ns:uid"),
        "SeanceRef": get_text(".//ns:seanceRef"),
        "SessionRef": get_text(".//ns:sessionRef"),
        "DateSeance": get_text(".//ns:dateSeance"),
        "DateSeanceJour": get_text(".//ns:dateSeanceJour"),
        "NumSeanceJour": get_text(".//ns:numSeanceJour"),
        "NumSeance": get_text(".//ns:numSeance"),
        "TypeAssemblee": get_text(".//ns:typeAssemblee"),
        "Legislature": get_text(".//ns:legislature"),
        "Session": get_text(".//ns:session"),
        "NomFichierJO": get_text(".//ns:nomFichierJo"),
        "President": get_text(".//ns:presidentSeance"),
    }

    # ÉTAPE 1 : Créer le mapping paragraphe -> point (contexte local)
    paragraphe_to_point = _creer_mapping_points(root, ns)

    # ÉTAPE 2 : Parcourir le contenu et extraire tous les paragraphes
    contenu = root.find("ns:contenu", ns)
    if contenu is None:
        print(f"⚠️ Pas de contenu trouvé dans {fichier_xml}")
        return pd.DataFrame()

    rows = []
    contexte_global = _initialiser_contexte()

    for elem in contenu.iter():
        tag = elem.tag.split("}")[-1]
        code_grammaire = elem.attrib.get("code_grammaire", "")
        code_style = elem.attrib.get("code_style", "")

        # Mettre à jour le contexte global
        _mettre_a_jour_contexte(elem, contexte_global, ns)

        # Traiter les paragraphes
        if tag == "paragraphe":
            row = _extraire_paragraphe(
                elem, metadata, contexte_global, paragraphe_to_point, ns
            )
            if row:
                rows.append(row)

    return pd.DataFrame(rows)


def _creer_mapping_points(root, ns):
    """
    Crée un mapping entre les paragraphes et leurs points parents
    pour récupérer le contexte local (sujet du point, valeur ODJ).
    """
    paragraphe_to_point = {}

    for point in root.findall(".//ns:point", ns):
        # Informations du point
        sujet_elem = point.find("ns:texte", ns)
        sujet_texte = (
            sujet_elem.text.strip()
            if sujet_elem is not None and sujet_elem.text
            else ""
        )
        valeur_ptsodj = point.attrib.get("valeur_ptsodj", "")

        # Récupérer d'autres métadonnées du point si disponibles
        point_id = point.attrib.get("id", "")
        point_type = point.attrib.get("type", "")

        point_info = {
            "sujet": sujet_texte,
            "valeur_odj": valeur_ptsodj,
            "point_id": point_id,
            "point_type": point_type,
        }

        # Associer tous les paragraphes de ce point (recherche récursive)
        for para in point.findall(".//ns:paragraphe", ns):
            para_id = para.attrib.get("id_syceron", "")
            if para_id:  # Seulement si on a un ID valide
                paragraphe_to_point[para_id] = point_info
            else:
                # Fallback : utiliser l'objet Python comme clé
                paragraphe_to_point[id(para)] = point_info

    return paragraphe_to_point


def _initialiser_contexte():
    """Initialise la structure de contexte global"""
    return {
        "titre_general": "",
        "sous_titre": "",
        "contexte_stack": [],
        "section_courante": "",
    }


def _mettre_a_jour_contexte(elem, contexte, ns):
    """
    Met à jour le contexte global basé sur l'élément courant.
    Gère différents niveaux hiérarchiques de titres.
    """
    code_grammaire = elem.attrib.get("code_grammaire", "")
    code_style = elem.attrib.get("code_style", "")

    texte_elem = elem.find("ns:texte", ns)
    texte = (
        texte_elem.text.strip() if texte_elem is not None and texte_elem.text else ""
    )

    if not texte:
        return

    # Titre principal
    if code_grammaire == "TITRE_TEXTE_DISCUSSION" and code_style == "Titre":
        contexte["titre_general"] = texte
        contexte["contexte_stack"] = [texte]

    # Sous-titre
    elif (
        code_grammaire == "SOUS_TITRE_TEXTE_DISCUSSION"
        and code_style == "Sous-tit_info"
    ):
        contexte["sous_titre"] = texte
        if len(contexte["contexte_stack"]) == 1:
            contexte["contexte_stack"].append(texte)
        else:
            contexte["contexte_stack"] = [contexte["titre_general"], texte]

    # Autres types de sections
    elif code_grammaire in ["TITRE_RUBRIQUE", "SOUS_TITRE_RUBRIQUE"]:
        if "SOUS" in code_grammaire:
            if len(contexte["contexte_stack"]) >= 2:
                contexte["contexte_stack"][1] = texte
            else:
                contexte["contexte_stack"].append(texte)
        else:
            contexte["contexte_stack"] = [texte]

    # Section courante pour d'autres cas
    elif code_style in ["Titre", "Sous-tit", "Sous-tit_info"]:
        contexte["section_courante"] = texte


def _extraire_paragraphe(elem, metadata, contexte_global, paragraphe_to_point, ns):
    """
    Extrait toutes les informations d'un paragraphe.
    Combine contexte global et contexte local (point).
    """
    texte_elem = elem.find("ns:texte", ns)
    if texte_elem is None:
        return None

    # Extraction du texte avec gestion des balises internes
    texte = "".join(texte_elem.itertext()).strip()
    if not texte:  # Skip les paragraphes vides
        return None

    stime = texte_elem.attrib.get("stime", "")

    # Informations orateur
    nom, qualite, id_orateur = _extraire_orateur(elem, ns)

    # Récupérer le contexte du point parent (si disponible)
    para_id = elem.attrib.get("id_syceron", "")
    point_info = paragraphe_to_point.get(para_id) or paragraphe_to_point.get(
        id(elem), {}
    )

    # Déterminer la source d'extraction
    source_extraction = "point" if point_info else "global"

    # Construire le contexte hiérarchique complet
    contexte_hierarchique = " > ".join(filter(None, contexte_global["contexte_stack"]))

    return {
        **metadata,
        # Contexte global
        "Titre_general": contexte_global["titre_general"],
        "Sous_titre": contexte_global["sous_titre"],
        "Contexte_hierarchique": contexte_hierarchique,
        "Section_courante": contexte_global["section_courante"],
        # Contexte local du point
        "Sujet_point": point_info.get("sujet", ""),
        "Valeur_ODJ": point_info.get("valeur_odj", ""),
        "Point_ID": point_info.get("point_id", ""),
        "Point_type": point_info.get("point_type", ""),
        # Informations du paragraphe
        "ID_paragraphe": para_id,
        "Ordre_seance": elem.attrib.get("ordre_absolu_seance", ""),
        "Code_grammaire": elem.attrib.get("code_grammaire", ""),
        "Code_style": elem.attrib.get("code_style", ""),
        "Code_parole": elem.attrib.get("code_parole", ""),
        "Role_debat": elem.attrib.get("roledebat", ""),
        # Informations de l'orateur
        "Nom_orateur": nom,
        "Qualite_orateur": qualite,
        "ID_orateur": id_orateur,
        # Contenu
        "stime": stime,
        "Texte": texte,
        # Métadonnées d'extraction
        "Source_extraction": source_extraction,
        # nope "Fichier_source": os.path.basename(elem.getroottree().getroot().attrib.get("fichier", ""))
    }


def _extraire_orateur(elem, ns):
    """Extrait les informations de l'orateur de manière robuste"""
    orateur_elem = elem.find(".//ns:orateur", ns)
    nom, qualite, id_orateur = "", "", ""

    if orateur_elem is not None:
        nom_elem = orateur_elem.find("ns:nom", ns)
        nom = nom_elem.text.strip() if nom_elem is not None and nom_elem.text else ""

        qualite_elem = orateur_elem.find("ns:qualite", ns)
        qualite = (
            qualite_elem.text.strip()
            if qualite_elem is not None and qualite_elem.text
            else ""
        )

        id_elem = orateur_elem.find("ns:id", ns)
        id_orateur = (
            id_elem.text.strip() if id_elem is not None and id_elem.text else ""
        )

    return nom, qualite, id_orateur


def traiter_dossier_xml(dossier_path: str, pattern: str = "*.xml") -> pd.DataFrame:
    """
    Traite tous les fichiers XML d'un dossier et retourne un DataFrame consolidé.
    """
    fichiers_xml = glob.glob(os.path.join(dossier_path, pattern))

    if not fichiers_xml:
        print(f"❌ Aucun fichier XML trouvé dans {dossier_path}")
        return pd.DataFrame()

    tous_dataframes = []
    stats = {"success": 0, "errors": 0, "total_rows": 0}

    print(f"🔄 Traitement de {len(fichiers_xml)} fichiers XML...")

    for fichier in fichiers_xml:
        nom_fichier = os.path.basename(fichier)
        print(f"   📄 {nom_fichier}...", end=" ")

        try:
            df = extraire_donnees_assemblee(fichier)
            if not df.empty:
                tous_dataframes.append(df)
                stats["success"] += 1
                stats["total_rows"] += len(df)
                print(f"✅ {len(df)} lignes")
            else:
                print("⚠️ Vide")

        except Exception as e:
            stats["errors"] += 1
            print(f"❌ Erreur: {str(e)}")

    print(f"\n📊 Statistiques:")
    print(f"   ✅ Fichiers traités avec succès: {stats['success']}")
    print(f"   ❌ Erreurs: {stats['errors']}")
    print(f"   📝 Total de lignes extraites: {stats['total_rows']}")

    if tous_dataframes:
        df_final = pd.concat(tous_dataframes, ignore_index=True)

        # Statistiques sur la répartition des sources
        if "Source_extraction" in df_final.columns:
            source_stats = df_final["Source_extraction"].value_counts()
            print(f"\n🎯 Répartition des sources:")
            for source, count in source_stats.items():
                print(
                    f"   {source}: {count} paragraphes ({count / len(df_final) * 100:.1f}%)"
                )

        return df_final
    else:
        return pd.DataFrame()


def analyser_extraction(df: pd.DataFrame):
    """
    Analyse les résultats de l'extraction pour identifier les différences.
    """
    if df.empty:
        print("❌ DataFrame vide")
        return

    print(f"\n📊 ANALYSE DE L'EXTRACTION")
    print(f"Total de paragraphes: {len(df)}")

    # Répartition par source
    if "Source_extraction" in df.columns:
        print(f"\n🔍 Répartition par source:")
        source_counts = df["Source_extraction"].value_counts()
        for source, count in source_counts.items():
            print(f"   {source}: {count} ({count / len(df) * 100:.1f}%)")

    # Paragraphes avec vs sans contexte de point
    if "Sujet_point" in df.columns:
        avec_point = df[df["Sujet_point"] != ""]
        sans_point = df[df["Sujet_point"] == ""]
        print(f"\n📝 Contexte des points:")
        print(f"   Avec sujet de point: {len(avec_point)}")
        print(f"   Sans sujet de point: {len(sans_point)}")

    # Types de codes grammaire les plus fréquents
    if "Code_grammaire" in df.columns:
        print(f"\n🏷️ Top 5 des codes grammaire:")
        top_codes = df["Code_grammaire"].value_counts().head()
        for code, count in top_codes.items():
            print(f"   {code or '(vide)'}: {count}")


# Exemple d'utilisation
# if __name__ == "__main__":

# # Pour un seul fichier
# print("🚀 Test sur un fichier unique...")
# df = extraire_donnees_assemblee("CRSANR5L16S2024O1N220.xml")

# # Analyse des résultats
# analyser_extraction(df)

# # Sauvegarde
# if not df.empty:
#     df.to_csv("assemblee_debat_hybride_ameliore.csv",
#               index=False,
#               quoting=csv.QUOTE_ALL,
#               encoding='utf-8')
#     print(f"\n✅ Fichier CSV généré : assemblee_debat_hybride_ameliore.csv")

# # Pour un dossier entier (décommenter si besoin)
# df_complet = traiter_dossier_xml("../data/16-xml/compteRendu/")
# analyser_extraction(df_complet)

In [9]:
df_temp = extraire_donnees_assemblee("CRSANR5L16S2024O1N220.xml")
df_temp.to_csv("V4.csv")

In [1]:
%pip install xmltodict

  Obtaining dependency information for xmltodict from https://files.pythonhosted.org/packages/d6/45/fc303eb433e8a2a271739c98e953728422fa61a3c1f36077a49e395c972e/xmltodict-0.14.2-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [12]:
#import module
import xmltodict

In [16]:
#open the file
fileptr = open("CRSANR5L16S2024O1N220.xml","r")


In [17]:
#read xml content from the file
xml_content= fileptr.read()

In [20]:
my_ordered_dict = xmltodict.parse(xml_content,
                                  # process_namespaces=True
                                  )

In [21]:
my_ordered_dict

{'compteRendu': {'@xmlns': 'http://schemas.assemblee-nationale.fr/referentiel',
  'uid': 'CRSANR5L16S2024O1N220',
  'seanceRef': 'RUANR5L16S2024IDS28385',
  'sessionRef': 'SCR5A2024O1',
  'metadonnees': {'dateSeance': '20240530213000000',
   'dateSeanceJour': 'jeudi 30 mai 2024',
   'numSeanceJour': '3',
   'numSeance': '220',
   'typeAssemblee': 'AN',
   'legislature': '16',
   'session': 'Session ordinaire 2023-2024',
   'nomFichierJo': '20240220',
   'validite': 'valide',
   'etat': 'complet',
   'diffusion': 'public',
   'version': 'avant_JO',
   'environnement': 'PROD',
   'heureGeneration': '2024-06-11T16:46:09.000+02:00',
   'sommaire': {'presidentSeance': {'@id_syceron': '3488040',
     '#text': 'Présidence de Mme\xa0Élodie Jacquier-Laforge'},
    'sommaire1': [{'@valeur_pts_odj': '1',
      'titreStruct': {'@id_syceron': '3488044',
       'intitule': 'Prise en charge intégrale des soins liés au traitement du cancer du sein',
       'sousIntitule': '0'},
      'para': {'@id_syc

In [32]:
# Essayer de parser ça si plus à l'aise, mais mmmeeehhhhhhh